# Analyze finetunned YOLO Models

Run the `run_yolo_detailed_testing_report.py` script on multiple YOLO models, collect metrics, and compare results.


In [ ]:
# Base directories
# Detect environment: Colab or local

import os
from pathlib import Path


IS_COLAB = 'COLAB_GPU' in os.environ or os.path.exists('/content')

USE_WANDB = False  # Set to False to disable W&B logging



if IS_COLAB:
    #Mount Google Drive if not already mounted
    from google.colab import drive
    drive.mount('/content/Drive', force_remount=True)
    # Running in Google Colab
    BASE_DIR = Path('/content/Drive/MyDrive/ksu_yolo_2025/computer_vision_yolo')
    
    # Configure W&B API key
    if USE_WANDB:
        # In Colab, get API key from secrets
        from google.colab import userdata
        wandb_api_key = userdata.get('wandb_api_key')
        os.environ['WANDB_API_KEY'] = wandb_api_key
        print('✓ W&B API key loaded from Colab secrets')

    DATASET_BASE_DIR = Path('/computer_vision_yolo')

else:
    # Running locally
    BASE_DIR = Path.cwd().parent
    if USE_WANDB:
        print('✓ Running locally - W&B will use existing login or prompt')
    
    DATASET_BASE_DIR = Path.cwd().parent


In [ ]:
#  ! cd /content/Drive/MyDrive/ksu_yolo_2025 && git clone https://github.com/m3mahdy/computer_vision_yolo

In [ ]:
# ! cd {BASE_DIR} && pip install -r requirements.txt --quiet

In [ ]:
# download limited dataset
# !mkdir {DATASET_BASE_DIR}
# !cd {BASE_DIR}/dataset && cp 8_download_extract_other_datasets.py {DATASET_BASE_DIR} && cd {DATASET_BASE_DIR} && python 8_download_extract_other_datasets.py


In [ ]:

# 1. Set Up Environment and Install Dependencies
import os
import sys
from pathlib import Path


import torch
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Image as IPImage

# Configure matplotlib for notebook
%matplotlib inline
matplotlib.rcParams['figure.max_open_warning'] = 50

print(f"Python: {sys.version}")
print(f"Torch version: {torch.__version__}")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")


IS_COLAB = 'COLAB_GPU' in os.environ or os.path.exists('/content')
if IS_COLAB:
    # Running in Google Colab
    sys.path.append("/content/Drive/MyDrive/ksu_yolo_2025/computer_vision_yolo/yolo_test")

else:
    # Running locally
    sys.path.append("/computer_vision_yolo/yolo_test")



PROJECT_ROOT = BASE_DIR
SCRIPT_PATH = PROJECT_ROOT / "yolo_test" / "run_yolo_validation_report.py"

# Add project root to path for imports
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
if str(PROJECT_ROOT / "yolo_test") not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT / "yolo_test"))

print(f"Project root: {PROJECT_ROOT}")
print(f"Script path: {SCRIPT_PATH}")


# Dataset Selection
# Option 1: Full dataset (~100k images) - for final optimization: "bdd100k_yolo"
# Option 2: Limited dataset (representative samples) - for quick tuning: "bdd100k_yolo_limited"
DATASET_NAME = 'bdd100k_yolo_tiny'  # 'bdd100k_yolo' or 'bdd100k_yolo_limited'
DATASET_SPLT = 'test'  # 'train', 'val', or 'test'

BATCH_SIZE = 256

# 2. Import validation functions from script
from run_yolo_detailed_testing_report import run_validation_pipeline, visualize_predictions
print("✓ Successfully imported validation functions")

# 3. Method Loop over models, run validation, and collect metrics
results_summary = []
validation_results = {}

def test_model(models_configs):

    for cfg in models_configs:
        print("=" * 80)
        print(f"Running model: {cfg['name']} | dataset={DATASET_NAME} | split={DATASET_SPLT} | IoU={cfg['iou']} | Include Training Exposure: {cfg['include_training_exposure']}")
        print("=" * 80)

        try:
            result = run_validation_pipeline(
                model_name=cfg["name"],
                dataset_name=DATASET_NAME,
                split=DATASET_SPLT,
                iou_threshold=cfg["iou"],
                base_dir=PROJECT_ROOT,
                use_wandb=True,
                save_reports=True,
                batch_size=BATCH_SIZE,
                include_training_exposure_analysis=cfg["include_training_exposure"],
                dataset_path=DATASET_BASE_DIR / DATASET_NAME,
            )

            validation_results[cfg["name"]] = result

            overall = result["metrics"]["overall"]
            yolo_overall = result["metrics"]["yolo_metrics"]

            results_summary.append({
                "model_name": cfg["name"],
                "dataset": DATASET_NAME,
                "split": DATASET_SPLT,
                "iou": cfg["iou"],
                "precision_confusion": overall["precision"],
                "recall_confusion": overall["recall"],
                "f1_confusion": overall["f1"],
                "precision_yolo": yolo_overall["precision"],
                "recall_yolo": yolo_overall["recall"],
                "map50": yolo_overall["map50"],
                "map50_95": yolo_overall["map50_95"],
                "params_m": result["model_info"]["params"] / 1e6,
                "size_mb": result["model_info"]["size(MB)"],
                "fps": result["metrics"]["fps"],
                "status": "ok",
                "run_dir": str(result["run_dir"]),
            })

        except Exception as e:
            print(f"⚠️ Model {cfg['name']} failed: {e}")
            import traceback
            traceback.print_exc()
            results_summary.append({
                "model_name": cfg["name"],
                "dataset": DATASET_NAME,
                "split": DATASET_SPLT,
                "iou": cfg["iou"],
                "status": "error",
            })



In [ ]:
# 4. Select model configurations to test

MODEL_CONFIGS = [

    {"name": "yolov8n_finetuned_1", "iou": 0.5, "include_training_exposure": True},
]

test_model(MODEL_CONFIGS)


## Test YOLOv8 Models

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov8n", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov8s", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov8m", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov8l", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov8x", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

## Test YOLOv9 Models

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov9t", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov9s", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov9m", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov9c", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov9e", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

## Test YOLOv10 Models

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov10n", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov10s", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov10m", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolov10x", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

## Test YOLO11 Models

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolo11n", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolo11s", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolo11m", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolo11x", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

## Test YOLO12 Models

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolo12n", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolo12s", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolo12m", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)

In [ ]:
MODEL_CONFIGS = [
    {"name": "yolo12x", "iou": 0.5, "include_training_exposure": False},
]

test_model(MODEL_CONFIGS)